<a href="https://colab.research.google.com/github/guthi1/mooc-exercises/blob/daffy/project/solution/dev/dt_yolo_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [2]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [3]:
from google.colab import drive
drive.mount('/content/drive')
with directory("/content/drive/My Drive/duckietown_dataset"):
  print(os.getcwd())
  print(os.listdir())

  with open("./test.txt", "w") as f:  # please verify that you did get a new file in your Drive
    f.write("Hello, world!")
os.chdir('/content/drive/My Drive/duckietown_dataset')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/duckietown_dataset
['images', 'annotation', 'frames', 'labels', 'yolov5', 'SENTINEL', 'minigrid-empty-5x5-image-dqn-2-100000', 'test.txt']


In [3]:

if not os.path.exists("SENTINEL"):
  prun("mkdir duckietown_dataset")
  prun("mv train duckietown_dataset && mv val duckietown_dataset")

# Next, we will clone Yolov5 into your google drive. 

In [4]:
!git clone https://github.com/duckietown/yolov5.git -b dt-obj-det
!cd yolov5 && pip3 install -r requirements.txt
!pip3 install torch==1.11 torchvision==0.12.0
if not os.path.exists("SENTINEL"):
  run("mv duckietown_dataset yolov5")
!touch SENTINEL

fatal: destination path 'yolov5' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 750.6 MB 16 kB/s 
     |████████████████████████████████| 21.0 MB 1.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0+cu116
    Uninstalling torch-1.13.0+cu116:
      Successfully uninstalled torch-1.13.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0+cu116
    Uninstalling torchvision-0.14.0+cu116:
      Successfully uninstalled torchvision-0.14.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.11.0 whi

# And we're ready to train! This step will take a hour or so.

Notice that we're only training for 10 epochs. That's probably not enough!

In [ ]:
!mv yolov5/best.pt yolov5/best_old.pt
!cd yolov5 && pip3 install -r requirements.txt && python3 train.py --img 416 --batch 16 --epochs 10 --data duckietown.yaml --weights yolov5s.pt

import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")

# Next, we can upload your model to Duckietown's cloud!

In [8]:
!pip3 install git+https://github.com/duckietown/lib-dt-mooc-2021

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/duckietown/lib-dt-mooc-2021 to /tmp/pip-req-build-c225gedq
  Running command git clone -q https://github.com/duckietown/lib-dt-mooc-2021 /tmp/pip-req-build-c225gedq
     |████████████████████████████████| 142 kB 14.7 MB/s 
  Created wheel for dt-mooc-2021: filename=dt_mooc_2021-0.0.0-py3-none-any.whl size=6447 sha256=a54d62333cbc4a23297e4022408f95e9ef54e61271f93c1f2e82b9e46283f6c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-wdaq5b7v/wheels/d5/df/21/aa957c23320a8241d9197289a987390c37adce3463c5ef9df8
  Created wheel for dt-data-api-daffy: filename=dt_data_api_daffy-1.1.3-py3-none-any.whl size=13714 sha256=3001e885fea638acea327569ca287250cadaa92736ac1999c872ed0f9ce34b13
  Stored in directory: /root/.cache/pip/wheels/38/ce/fb/5a988e5766d210a7586879219893e14a85ebca9f179ee91041
  Created wheel for dt-authentication-daffy: filename=dt_authentication_daffy-0.1.1

In [9]:
from dt_mooc.cloud import Storage
import sys
import torch

def select_device(device='', batch_size=None):
    import torch
    # device = 'cpu' or '0' or '0,1,2,3'
    cpu = device.lower() == 'cpu'
    if cpu:
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # force torch.cuda.is_available() = False
    elif device:  # non-cpu device requested
        os.environ['CUDA_VISIBLE_DEVICES'] = device  # set environment variable
        assert torch.cuda.is_available(), f'CUDA unavailable, invalid device {device} requested'  # check availability

    cuda = not cpu and torch.cuda.is_available()

    return torch.device('cuda:0' if cuda else 'cpu')

sys.path.insert(0, './yolov5')
model = torch.load("./yolov5/best.pt", map_location=select_device("cpu"))['model'].float()  # load to FP32
model.to(select_device("cpu")).eval()

# storage = Storage("YOUR TOKEN HERE")

# storage.upload_yolov5("yolov5", model, "./yolov5/best.pt")

Model(
  (model): Sequential(
    (0): Focus(
      (conv): Conv(
        (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (cv3): Conv(
       

# Done!

We're done training! You can go back to the `Training` notebook